In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool
from langchain_core.tools import tool
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_google_vertexai.vision_models import VertexAIImageGeneratorChat
from langchain_core.messages import AIMessage, HumanMessage
from crewai import Agent, Task, Crew, Process
import datetime
import base64
import io
from PIL import Image
from IPython.display import Markdown

In [ ]:
llm = ChatVertexAI(
    model_name="gemini-1.5-flash"
)

In [ ]:
ddg_wrapper = DuckDuckGoSearchAPIWrapper(region="uk-en", time="w", max_results=100)
search = DuckDuckGoSearchResults(api_wrapper=ddg_wrapper, backend="text")

search_tool = Tool(
    name="search",
    description="Search the internet for relevant information",
    func=search.run,
)

In [ ]:
@tool("date_tool", return_direct=True)
def date_tool():
    """Today's date"""
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    return f"Today's date is {today}."

@tool("arxiv_tool", return_direct=False)
def arxiv_tool():
    """Look up latest academic journal articles."""
    loader = AsyncHtmlLoader("https://arxiv.org/list/cs/new?skip=0&show=50")
    docs = loader.load()
    html2text = Html2TextTransformer(ignore_links=False)
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed

@tool("url_tool", return_direct=False)
def url_tool(urls: list[str]):
    """Download webpage content directly from URLs."""
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    html2text = Html2TextTransformer(ignore_links=False)
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed

In [ ]:
data_science_researcher = Agent(
    role="Data Science Researcher",
    goal="Monitor and analyse the latest development in Data Science by researching papers posted on the HuggingFace or ArXiv. "
         "Presents complex concepts in layman terms so that everyone understands. ",
    backstory="Being an experienced data science researcher, this agent has extensive experience in "
              "conducting internet serach to identify latest trends in Data Science, Artificial Intelligence, Machine Learning. "
              "This agent always reads journal papers, articles, websites to stay up-to-date. "
              "This agent MUST use the arxiv_tool to read latest academic papers. "
              "This agent will always pass on the findings alongside the source URLs. "
              "The research must be published within the previous seven days. Today's date can be found using tool. "
              "Use tools if needed and give final answer once the  right information has been collected. ",
    llm=llm,
    verbose=True,
    tools = [
        date_tool,
        search_tool,
        arxiv_tool,
        url_tool
    ],
    allow_delegation=False,
)

social_media_writer = Agent(
    role="Social Media Writer",
    goal="Create trending social media post on LinkedIn on Data Science, AI and Machine Learning topics "
         "based on findings from the Data Science Researcher.",
    backstory="This agent specialises in writing trending LinkedIn posts, "
              "understands how best to present information to both technical and non-technical users. "
              "This agent can instruct coworkers to provide relevant information to accomplish tasks. "
              "You will use tools if needed and give final answer once you have collected the right information. ",
    llm=llm,
    verbose=True,
    tools = [
        url_tool
    ],
    allow_delegation=True,
)

In [ ]:
data_science_research_task = Task(
    description=(
        "Continuously monitor latest development on these topic: {topic}. "
        "You will read the academic journal articles and return ONE article which is most relevant. "
        "The article you return must be an academic journal paper relating to one of the chosen topics. "
        "The URL must be extracted using output of the tool. "
    ),
    expected_output=(
        "Summary of findings and the corresponding source URL, which links to the original article. "
    ),
    agent=data_science_researcher,
)

draft_post_task = Task(
    description=(
        "Draft a trending LinkedIn post on one of these topics: {topic}. "
        "Write in simple English, so gthat it can be understood by both technical and non-technical audiences."
        "Summarise what the developments are and suggest how the they can be used in real-world scenario, solving actual business problems. "
        "The source URLs provided by the Data Science Researcher must be included in the draft post. "
        "Placeholders such as [NAME] must not be used. "
        "You can add emojis to make the post more interesting. "
        "You should also use suitable hashtags to make the post more engaging. "
    ),
    expected_output=(
        "An interesting and engaging social media post on LinkedIn. "
        "You must include exactly one source URL which links to the journal article. "
    ),
    agent=social_media_writer,
    context=[
        data_science_research_task,
    ],
)

verification_task = Task(
    description=(
        "Download contents from the source URL using tool to ensure it is the correct link to use, given the draft LinkedIn post. "
        "The link should lead to the full article or the journal paper. It should not simply be a landing page without details. "
        "If they do not seem to align, or the URL doesn't work, you must request this to be corrected. "
    ),
    expected_output=(
        "Verify that the content and source URL are correct and aligned with one another. URL must be in the draft post. "
    ),
    agent=social_media_writer,
    context=[
        draft_post_task,
    ],
)

image_prompt_task = Task(
    description=(
        "Based on the draft LinkedIn post, write a prompt to generate an image. "
    ),
    expected_output=(
        "An image prompt which is suits the contents of the draft post."
    ),
    agent=social_media_writer,
    context=[
        verification_task,
    ],
)

In [ ]:
# Define the crew with agents and tasks
crew = Crew(
    agents=[
        data_science_researcher,
        social_media_writer,
    ],
    tasks=[
        data_science_research_task,
        draft_post_task,
        verification_task,
        image_prompt_task,
    ],
    process=Process.hierarchical,
    manager_llm=llm,
    verbose=1,
    memory=True,
    full_output=True,
    embedder={
        "provider": "vertexai",
        "config":{
            "model": 'textembedding-gecko'
        }
    }
)

In [ ]:
result = crew.kickoff(
    inputs={
        "topic": "Data Science, Machine Learning, Generative AI, LLM, NLP, Graph Theory, Multiagent System"
    }
)

In [ ]:
Markdown(draft_post_task.output.exported_output)

# Vertex AI Imagen

Draw a picture to go with the content. Reference URL: https://cloud.google.com/vertex-ai/generative-ai/docs/image/overview

In [ ]:
def generate_image(prompt:str, filepath:str):
    """Generate an image based on the given prompt and save it locally"""
    
    generator = VertexAIImageGeneratorChat()
    messages = [HumanMessage(content=[prompt])]
    response = generator.invoke(messages)
    generated_image = response.content[0]


    # Parse response object to get base64 string for image
    img_base64 = generated_image["image_url"]["url"].split(",")[-1]

    # Convert base64 string to Image
    img = Image.open(io.BytesIO(base64.decodebytes(bytes(img_base64, "utf-8"))))

    # view Image
    img.save(filepath, "JPEG")

In [ ]:
generate_image(image_prompt_task.output.exported_output, "image.jpg")